# To calibration the polarizer and analyzer angle

In [2]:
from cProfile import label
from cmath import exp, pi, tan, sin, sqrt,acos,asin
from re import A
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as npy
from scipy.signal import savgol_filter
from matplotlib.pyplot import MultipleLocator
from iminuit import Minuit
import iminuit
from iminuit.cost import LeastSquares
# import latex
# plt.style.reload_library()
plt.style.use(['science'])
# plt.style.use(['science','ieee'])
# plt.style.use(['science','no-latex'])

## The dark current

In [3]:
def currentDark(filename):
    inputfile=open(filename,'r')
    measurement_dark=[]
    monitor_dark=[]
    while(1):
        try:
            temp=next(inputfile).split()
            if(temp[4]=='dark'):
                measurement_dark.append(1e6*float(temp[2]))
            elif(temp[4]=='dark1'):
                monitor_dark.append(1e6*float(temp[2]))
        except StopIteration:
            break
    dark_meansurement=npy.mean(npy.array(measurement_dark))
    dark_monitor=npy.mean(npy.array(monitor_dark))
    return dark_meansurement,dark_monitor
dark_measurement,dark_monitor = currentDark('measurement_dark.txt')

## Define class and fit function

In [4]:
def calculate_index(phi,delta):
    phi=float(phi*pi/180)
    delta=float(delta*pi/180)
    anglei=65/180*pi
    rho=tan(phi)*exp(delta*1j)
    n1=1.
    # print('rho',rho)
    epsilon= (n1**2) * (sin(anglei)**2) * ( 1+(tan(anglei)**2)*((1-rho)/(1+rho))**2) 
    ep_re=epsilon.real
    ep_im=epsilon.imag
    n2=(1./sqrt(2.) * sqrt( sqrt(ep_re**2+ep_im**2) +ep_re)).real
    k2=(1./sqrt(2.) * sqrt( sqrt(ep_re**2+ep_im**2) -ep_re)).real
    return n2,k2

def intensityFun(t,a0,a1,a2,w): #define fit function
    phi=w*t
    # return a0+a1*npy.cos(2*phi)
    return a0+a1*npy.cos(2*phi)+ a2*npy.sin(2*phi)

class dataVar:
    def __init__(self,angleP):
        self.angleP=angleP
        self.current=[]
        self.time=[]
        self.angleA=[]

def reader(filename,angleP,use):
    data=dataVar(angleP)
    inputfile=open(filename,'r')
    i=0
    while(1):
        try:
            temp=next(inputfile).split()
            if(temp[4]!=('P'+str(angleP))):
                continue
            data.time.append(i*0.5)
            if(use=='measurement'):
                data.current.append(1e6*float(temp[2])-dark_measurement)
            elif(use=='monitor'):
                data.current.append(1e6*float(temp[2])-dark_monitor)
            i=i+1
        except StopIteration:
            break
    return data

## Calculate a series of $\alpha$ and $\beta$ for future fit

In [7]:
a=[3,0,3,3,0,2,1,1,1,2,2]  # the delay time of two Ammeters 
w=0.035   # this is the estimated angular velocity
alpha=[]
beta=[]
angleP=[]

for ip in range(11):
    angleA=[]
    angleP_temp=ip*9    #in degree
    angleP.append(angleP_temp/180*pi)
    window=10
    
    data_measurement=reader('measurement_caliAP.txt',angleP_temp,'measurement')
    data_monitor=reader('monitor_caliAP.txt',angleP_temp,'monitor')

    length=len(data_monitor.current)-20
    # print(length)
    b=0
    data_monitor.time=data_monitor.time[0:length]
    data_monitor.current=data_monitor.current[0+a[ip]:length+a[ip]]
    data_measurement.time=data_measurement.time[0:length]
    data_measurement.current=data_measurement.current[0+b:length+b]

    signal=npy.array(data_measurement.current)/npy.array(data_monitor.current)
    signal=savgol_filter(signal,window,3)  # Filtering operation
    # average of the end current
    mean_end=npy.mean(signal[720:])
    end_time=0
    for t in range(200):
        if(signal[680+t]>=mean_end):
            if(ip==10):
                end_time=680+t-6
            elif(ip==8):
                end_time=680-t-9
            else:
                end_time=680+t
            break
        else:
            continue
    
    signal_cut=npy.array(signal[0:end_time])
    time_cut=npy.array(data_measurement.time[0:end_time])

    fig,ax=plt.subplots(2,1,sharex=True)
    ax[1].set_xlabel('Time (s)')
    ax[0].set_ylabel('Intensity')
    # ax[0].set_ylim()
    ax[0].plot(data_measurement.time,signal,label='Water 400nm',linewidth=1,color='orange')#,color='blue)
    ax[0].plot(time_cut,signal_cut,label='After cut',linestyle="--",color='k',linewidth=1)#,color='blue)
    ax[1].set_ylabel('Current (uA)')
    ax[1].plot(data_measurement.time,data_measurement.current,label='measurement')
    ax[1].plot(data_monitor.time,data_monitor.current,label='monitor')
    fig.suptitle(str(ip*9)+' Deg. Intensity Vs. Time')
    fig.subplots_adjust(hspace=0.1)
    fig.patch.set_facecolor('white')
    ax[0].legend()
    ax[1].legend()
    plt.close()
    fig.savefig('cali_figure/'+ str(ip*9)+ 'DegIntensityVsTime.png',dpi=1000)
    
    #time_cut and signal_cut is the x y we need to fit.
    time_cut=time_cut-0.5*(len(time_cut)-1)
    p0=[0,0,0,w]
    para,_=curve_fit(intensityFun,time_cut,signal_cut,p0)
    print(para)
    alpha.append(para[1]/para[0])
    beta.append(para[2]/para[0])
    signal_fit=intensityFun(time_cut,*para)
    ratio_fit=signal_cut/signal_fit
    fig,ax=plt.subplots(2,1,sharex=True)
    ax[1].set_xlabel('Time (s)')
    ax[0].set_ylabel('Intensity')
    ax[0].plot(time_cut,signal_cut,label='Ori. signal',linewidth=1,color='orange')#,color='blue)
    ax[0].plot(time_cut,signal_fit,label='Fitted signal',linestyle="--",color='k',linewidth=1)#,color='blue)
    ax[1].set_ylabel('Ratio')
    ax[1].plot(time_cut,ratio_fit,'.',label='Ori. Vs. Fitted',markersize=1)
    plt.fill_between(time_cut,npy.mean(ratio_fit)+npy.std(ratio_fit),npy.mean(ratio_fit)-npy.std(ratio_fit),alpha=0.3)
    ax[1].fill
    fig.suptitle(str(ip*9)+' Deg. Fit Result')
    ax[0].legend()
    ax[1].legend()
    fig.patch.set_facecolor('white')
    fig.savefig('cali_figure/Fit'+ str(ip*9)+ 'DegIntensityVsTime.png',dpi=1000)
    plt.close()
    

[7.1867248  3.76868412 5.48189846 0.03487247]
[8.51983087 3.99256522 6.83808382 0.03487583]
[9.61251862 3.3927148  8.28060567 0.03487171]
[10.36753798  2.97116362  9.19805908  0.03486678]
[10.68765852  2.00166691  9.7714324   0.03487788]
[10.53968209  1.20488751  9.75931981  0.03488302]
[9.96913037 0.57837988 9.27544289 0.03487996]
[ 8.99386526 -0.86568068  8.3093973   0.03488241]
[ 7.72797985 -3.65605806 -6.14649814  0.03488972]
[ 6.30913034 -1.93325576  5.47165149  0.03487979]
[ 4.87565185 -2.40094005  3.71111309  0.03488419]


## Fit the phi a b